## Etapa 1: Instalação das bibliotecas

In [0]:
!pip install tensorflow-gpu==2.0.0.alpha0

     |████████████████████████████████| 332.1MB 63kB/s 
     |████████████████████████████████| 3.0MB 44.6MB/s 
     |████████████████████████████████| 419kB 46.2MB/s 


## Etapa 2: Importação das bibliotecas

In [0]:
import time
import numpy as np
import tensorflow as tf

In [0]:
tf.__version__

'2.0.0-alpha0'

## Etapa 3: Pré-processamento

### Carregando a base de dados MNIST

In [0]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

### Normalização das imagens

In [0]:
X_train = X_train / 255.
X_test = X_test / 255.

In [0]:
X_train.shape

(60000, 28, 28)

### Mudança das dimensões

In [0]:
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

In [0]:
X_train.shape

(60000, 784)

## Etapa 4: Treinamento distribuído

### Definição do modelo normal (não distribuído)

In [0]:
model_normal = tf.keras.models.Sequential()
model_normal.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
model_normal.add(tf.keras.layers.Dropout(rate=0.2))
model_normal.add(tf.keras.layers.Dense(units=10, activation='softmax'))
model_normal.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

### Definição da estratégia distribuída

In [0]:
distribute = tf.distribute.MirroredStrategy()

INFO:tensorflow:Device is available but not used by distribute strategy: /device:CPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_CPU:0


### Definindo um modelo distribuído

In [0]:
with distribute.scope():
  model_distributed = tf.keras.models.Sequential()
  model_distributed.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
  model_distributed.add(tf.keras.layers.Dropout(rate=0.2))
  model_distributed.add(tf.keras.layers.Dense(units=10, activation='softmax'))
  model_distributed.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

### Comparação de velocidade entre o treinamento normal e o treinamento distribuído

In [0]:
starting_time = time.time()
model_distributed.fit(X_train, y_train, epochs=20, batch_size=128)
print("Distributed training: {}".format(time.time() - starting_time))

INFO:tensorflow:batch_all_reduce invoked for batches size = 4 with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
Epoch 1/20
469/469 [==============================] - 4s 7ms/step - loss: 0.4019 - sparse_categorical_accuracy: 0.8845
Epoch 2/20
469/469 [==============================] - 3s 6ms/step - loss: 0.1911 - sparse_categorical_accuracy: 0.9453
Epoch 3/20
469/469 [==============================] - 3s 6ms/step - loss: 0.1461 - sparse_categorical_accuracy: 0.9572
Epoch 4/20
469/469 [==============================] - 3s 6ms/step - loss: 0.1190 - sparse_categorical_accuracy: 0.9645
Epoch 5/20
469/469 [==============================] - 3s 6ms/step - loss: 0.1013 - sparse_categorical_accuracy: 0.9696
Epoch 6/20
469/469 [==============================] - 3s 6ms/step - loss: 0.0880 - sparse_categorical_accuracy: 0.9738
Epoch 7/20
469/469 [==============================] - 3s 6ms/step - loss: 0.0764 - sparse_categorical_accuracy: 0.9765
Ep

In [0]:
starting_time = time.time()
model_normal.fit(X_train, y_train, epochs=20, batch_size=128)
print("Normal training: {}".format(time.time() - starting_time))

Epoch 1/20
60000/60000 [==============================] - 2s 42us/sample - loss: 0.4000 - sparse_categorical_accuracy: 0.8862
Epoch 2/20
60000/60000 [==============================] - 3s 42us/sample - loss: 0.1903 - sparse_categorical_accuracy: 0.9449
Epoch 3/20
60000/60000 [==============================] - 2s 39us/sample - loss: 0.1432 - sparse_categorical_accuracy: 0.9582
Epoch 4/20
60000/60000 [==============================] - 2s 41us/sample - loss: 0.1205 - sparse_categorical_accuracy: 0.9645
Epoch 5/20
60000/60000 [==============================] - 2s 40us/sample - loss: 0.1009 - sparse_categorical_accuracy: 0.9696
Epoch 6/20
60000/60000 [==============================] - 2s 41us/sample - loss: 0.0909 - sparse_categorical_accuracy: 0.9732
Epoch 7/20
60000/60000 [==============================] - 2s 40us/sample - loss: 0.0798 - sparse_categorical_accuracy: 0.9752
Epoch 8/20
60000/60000 [==============================] - 2s 40us/sample - loss: 0.0710 - sparse_categorical_accuracy: